# 문제정의
## 고객별 신용등급 분류
## Column : Credit_Score - 타깃 변수(y), 신용등급(1=Bad, 2=Standard, 3=Good)
## 타깃(라벨, y값) : Credit_Score

In [10]:
#----------------------
# 라이브러리 가져오기
#----------------------

In [11]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')



In [12]:
#----------------------
# EDA(데이터 전처리)
#----------------------

In [13]:
train.head()
train.tail()
train.sample(5)

# Credit_score가 인코딩이 되어있는 상태라 y값을 따로 정의할 필요는 없을 듯!

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_Mix,...,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
8458,9.0,20.0,5.0,35.742630,266.0,Yes,59.781488,420.982451,2,Standard,...,29.0,29482.060,8.0,3.0,9.0,2.0,2395.838333,14.09,668.50,28.819894
4479,24.0,18.0,11.0,23.553740,36.0,NM,139.599486,183.436346,1,Bad,...,38.0,20372.430,10.0,5.0,17.0,7.0,1411.702500,19.96,3460.00,108.134418
9878,15.0,16.0,7.0,27.070771,192.0,Yes,53.295595,268.705176,1,Standard,...,29.0,8333.545,3.0,4.0,20.0,6.0,698.462083,10.29,1850.44,37.845438
5967,1.0,6.0,8.0,32.050181,368.0,No,228.150302,819.579087,3,Good,...,30.0,102215.610,2.0,5.0,10.0,1.0,8482.967500,9.90,79.67,60.567361
1384,49.0,18.0,11.0,36.814881,135.0,Yes,83.451699,297.775643,3,Bad,...,15.0,17157.470,10.0,7.0,22.0,6.0,1656.789167,23.79,3743.28,44.451575


In [14]:
train.shape, test.shape

((10000, 21), (10000, 20))

In [15]:
# train의 y값 확인
train['Credit_Score']

# 표형태로 볼려면
train[['Credit_Score']]

# 얼마나 카운팅되어있는지 확인
train[['Credit_Score']].value_counts()

Credit_Score
2               5237
1               2978
3               1785
Name: count, dtype: int64

In [16]:
# 범주형/수치형 구성 확인
train.info()
test.info()
# --> 결측치 X, 결측치 처리 필요 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Score              10000 non-null  int64  
 9   Credit_Mix                10000 non-null  object 
 10  Payment_Behaviour         10000 non-null  object 
 11  Age                       10000 non-null  float64
 12  Annual_Income             10000 non-null  float64
 13  Num_Bank_Accounts         10000 non-null  float64
 14  Num_Cre

In [17]:
# 이상치 확인
train.describe()
test.describe()

# 범주형 이상치 확인
train.describe(include='O')
test.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5167,4590,3498


In [18]:
# 라벨(정답, y값) 분리
# 학습시켜야하는 모델의 파라미터로 전달
# y_train = train.pop("Credit_Score")
y_train

y_train.value_counts()

Credit_Score
2    5237
1    2978
3    1785
Name: count, dtype: int64

In [19]:
# 기존 레이블 인코딩

# from sklearn.preprocessing import LabelEncoder
# cols = train.columns[train.dtypes == object] # 범주형 -> 수치형

# for col in cols:
#     le = LabelEncoder()

#     #피팅되는 경우가 안 맞는 경우 각 행마다 이동할 때 train,text행을 합치기
#     le.fit(pd.concat([train[col],test[col]] ,axis=0)) # axis=0 행합치기, axis=1 열합치기

#     #따로 분리
#     train[col] = le.transform(train[col])
#     test[col] = le.transform(test[col])

#### 인코딩대체방법(범주형 type -> category로 변경)

In [20]:
train.info()
# category로 Dtype(데이터 타입) 변경
train['Payment_of_Min_Amount'] = train['Payment_of_Min_Amount'].astype('category')
train['Credit_Mix'] = train['Credit_Mix'].astype('category')
train['Payment_Behaviour'] = train['Payment_Behaviour'].astype('category')

test['Payment_of_Min_Amount'] = train['Payment_of_Min_Amount'].astype('category')
test['Credit_Mix'] = train['Credit_Mix'].astype('category')
test['Payment_Behaviour'] = train['Payment_Behaviour'].astype('category')

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Mix                10000 non-null  object 
 9   Payment_Behaviour         10000 non-null  object 
 10  Age                       10000 non-null  float64
 11  Annual_Income             10000 non-null  float64
 12  Num_Bank_Accounts         10000 non-null  float64
 13  Num_Credit_Card           10000 non-null  float64
 14  Interes

In [21]:
#문자형 -> 숫자형으로 변환
train
test

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Mix,Payment_Behaviour,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,4.0,8.0,2.0,33.444469,239.0,Yes,104.467859,503.976904,Bad,High_spent_Medium_value_payments,54.0,43754.320,5.0,7.0,7.0,1.0,3938.193333,7.55,515.73,35.374569
1,31.0,19.0,14.0,27.762143,93.0,Yes,149.137974,219.372773,Bad,High_spent_Small_value_payments,40.0,18836.230,6.0,8.0,16.0,6.0,1280.685833,14.26,1310.56,49.557837
2,55.0,16.0,17.0,38.985106,179.0,Yes,21.862086,299.525207,Bad,Low_spent_Large_value_payments,27.0,9360.515,8.0,6.0,15.0,5.0,531.042917,11.92,2113.29,21.716999
3,47.0,20.0,8.0,25.577407,93.0,NM,170.009215,161.283266,Standard,High_spent_Small_value_payments,34.0,30386.200,7.0,8.0,21.0,9.0,2650.183333,17.53,4902.83,193.725852
4,26.0,11.0,7.0,36.580166,207.0,Yes,296.552469,177.231449,Bad,High_spent_Medium_value_payments,36.0,49187.070,6.0,6.0,25.0,6.0,3823.922500,12.04,1357.26,198.608332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,27.0,12.0,4.0,38.977305,383.0,No,39.695906,472.869011,Standard,High_spent_Medium_value_payments,32.0,38839.220,8.0,4.0,19.0,3.0,3272.601667,1.83,1203.51,64.695250
9996,22.0,19.0,2.0,25.582411,363.0,Yes,89.540202,360.819998,Standard,Low_spent_Small_value_payments,51.0,24251.490,8.0,4.0,14.0,1.0,1991.957500,10.53,1062.17,18.835550
9997,20.0,15.0,11.0,26.764418,233.0,Yes,53.574209,311.419464,Standard,High_spent_Medium_value_payments,45.0,15806.840,6.0,5.0,21.0,3.0,1140.236667,3.41,2580.63,29.029993
9998,60.0,19.0,8.0,35.262194,99.0,Yes,46.902804,389.691164,Standard,High_spent_Medium_value_payments,40.0,35462.520,8.0,9.0,31.0,4.0,2718.210000,13.14,2371.87,75.227032


In [22]:
#------------------------
# 검증 데이터 분할
#------------------------

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train,
    y_train, #target 위치
    test_size=0.2,
    random_state=0)

In [24]:
X_train, X_val, y_train, y_val
#train : 학습에 사용될, val : 검증에 사용될

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((8000, 20), (2000, 20), (8000,), (2000,))

#--------------------
# 머신러닝 학습 및 평가
#--------------------

## 머신러닝 학습

In [25]:
# 1. 랜덤포레스트 학습

# 범주형 데이터를 자료형만 바꿨을 때는 문제가 발생할 예정
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0) # 난수값고정 in->out 고정값
rf.fit(X_train, y_train) # 모델 학습
pred_rf=rf.predict_proba(X_val)
pred_rf # 결과 - ValueError: could not convert string to float: 'Yes'

ValueError: could not convert string to float: 'Yes'

In [26]:
#2. LIGHTGBM 학습
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1) # 
lgbmc.fit(X_train, y_train)
pred=lgbmc.predict(X_val)  # 예측 y값
pred_proba = lgbmc.predict_proba(X_val) # 예측 확률값

## 평가

In [27]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

roc_auc_ovr = roc_auc_score(y_val , pred_proba, multi_class='ovr') # pred 예측 확률값 [오답예측확률 , 정답예측확률]
roc_auc_ovo = roc_auc_score(y_val , pred_proba, multi_class='ovo') # pred 예측 확률값 [오답예측확률 , 정답예측확률]
accuracy = accuracy_score(y_val,pred) # pred 예측 y값 
f1 = f1_score(y_val,pred,average='macro') # pred 예측 y값 (클래스별(1,2,3정답값) 성능을 동등하게 반영)

print('roc_auc_ovr : ',  roc_auc_ovr)
print('roc_auc_ovo : ',  roc_auc_ovo)
print('accuracy : ',  accuracy)
print('f1 : ',  f1)

# OvR(One-vs-Rest): 각 클래스 vs 나머지 → 평균
# OvO(One-vs-One): 클래스 쌍별 AUC 평균

roc_auc_ovr :  0.8507625027884959
roc_auc_ovo :  0.8575703288828868
accuracy :  0.6985
f1 :  0.6777379561595467


#-------------------------
# 예측 파일 제출(.csv)
#-------------------------

In [ ]:
pred_submit=lgbmc.predict(test)
pred_submit

In [ ]:
submit_csv = pd.DataFrame({'pred':pred_submit})
submit_csv.to_csv("result.csv",index=False)
